In [20]:
%%file api_encryption.py
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

def generate_key_and_iv(password, salt):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32 + 16,  # Key length + IV length
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key_iv = kdf.derive(password.encode())
    return key_iv[:32], key_iv[32:]

def encrypt_file(file_path, password):
    with open(file_path, 'rb') as file:
        data = file.read()

    salt = os.urandom(16)
    key, iv = generate_key_and_iv(password, salt)

    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    encrypted_data = encryptor.update(data) + encryptor.finalize()

    with open(file_path + '.encrypted', 'wb') as file:
        file.write(salt + encrypted_data)

def decrypt_file(file_path, password):
    with open(file_path, 'rb') as file:
        salt = file.read(16)
        data = file.read()

    key, iv = generate_key_and_iv(password, salt)

    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    decrypted_data = decryptor.update(data) + decryptor.finalize()

    with open(file_path.replace('.encrypted', '_decrypted.txt'), 'wb') as file:
        file.write(decrypted_data)

# Replace 'file.txt' with your actual file name
file_to_encrypt = os.argv[1]

if os.argv[2] == 1:
    # Replace 'your_password_here' with your actual password
    print("Encryption mode, set a password : ",end='')
    password = input()
    encrypt_file(file_to_encrypt, password)
elif os.argv[2] == 0:
    # Replace 'your_password_here' with your actual password
    print("Decryption mode, password : ",end='')
    password = input()
    decrypt_file(file_to_encrypt + '.encrypted', password)


Writing api_encryption.py


In [ ]:
%%bash 
python3 api_encryption.py test.txt 1

In [28]:
import random
import string

# Define the characters you want to include in the random string
characters = string.ascii_letters + string.digits  # You can include punctuation or other characters if needed

# Generate a random string of 50 elements
random_string = ''.join(random.choice(characters) for _ in range(100))

print(random_string)

LbXkvWuvrTWuK1D1aKM7R4rAkP6iJefOJbyG3E67USlvtvh6Inh1GobJL3fENyWLFaYeIsYYXOQiOmdtp1uQsMR6fyIi5nA7DLs0
